In [1]:
import json
import joblib
from pathlib import Path

from econml.dml import LinearDML, DML
from catboost import CatBoostClassifier, CatBoostRegressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm

from sklearn.model_selection import GroupKFold, StratifiedGroupKFold

In [2]:
class GroupStratifiedKFold:
    def __init__(self, n_splits: int = 3, random_state: int = None):
        self.sgk = StratifiedGroupKFold(
            n_splits=n_splits,
            shuffle=True,
            random_state=random_state
        )

    def split(self, X, y, groups):
        return self.sgk.split(X, y, groups)

In [3]:
data_path = Path("../data")
file_name = "ToAnalysis_Winsorized_2015_2023_With_Profit_Asset.csv"
file_path = data_path / file_name

df = pd.read_csv(file_path, parse_dates=['start date'])
df.columns = [col.replace(" ", "_").lower() for col in df.columns]
df = df.sort_values(by=['ticker', 'start_date'])

df["log_tobin_q_winsor"] = np.log(df['tobin_q_winsor'])
df["year"] = df.start_date.dt.year
df["roa"] = df["net_income"].div(df["total_assets"])
df["log_total_assets"] = np.log(df['total_assets'])
df["log_leverage"] = np.log(df['leverage'])
df["log_tangible_assets"] = np.log(df['tangible_assets'])

df = df.assign(lag_log_tobin_q_winsor=df.groupby(by=['ticker']).log_tobin_q_winsor.shift(-1))

In [4]:
df['female_threshold'] = df['female_director_ratio'].mask((df['female_director_ratio']<= 0.1) & (df['female_director_ratio']> 0), 1)
df['female_threshold'] = df['female_threshold'].mask((df['female_director_ratio']<= 0.2) & (df['female_director_ratio']> 0.1), 2)
df['female_threshold'] = df['female_threshold'].mask((df['female_director_ratio']<= 0.3) & (df['female_director_ratio']> 0.2), 3)
df['female_threshold'] = df['female_threshold'].mask((df['female_director_ratio']> 0.3), 4)

In [5]:
year = pd.get_dummies(df['year'], dtype='int')
year = year.drop(2015, axis=1)

# 業種の固定効果
industry = pd.get_dummies(df['industry_name'], dtype='int')
industry = industry.drop("その他製品", axis=1)

In [6]:
# Lag Version
Y_cols = ['lag_log_tobin_q_winsor']

# Control 変数 (ROAは入れた方が予測性能がよくなる)
W_cols = [
    'board_size', 
    'log_firm_age', 
    'log_sales',
    'sales_growth', 
    'foreign_ownership', 
    'managerial_ownership',
    'log_tangible_assets', 
    'log_leverage',
    'roa',
    'net_income'
]

X_cols = []
T_cols = ['female_threshold']
G_cols = ['ticker']

mundlak_W = df.groupby(by='ticker')[W_cols + X_cols].transform("mean")
mundlak_W.columns = [f"{col}_mean" for col in mundlak_W.columns]

mundlak_T = df.groupby(by='ticker')[T_cols].transform("mean")
mundlak_T.columns = [f"{col}_mean" for col in mundlak_T.columns]

# コントロール変数
W = df[W_cols]
W = W.join(mundlak_T).join(mundlak_W)
W = W.join(year).join(industry)

# 説明変数
X = df[X_cols]

# 出力
Y = df[Y_cols]

# 介入
T = df[T_cols]

# Groups
G = df['ticker']

tmp = pd.concat((W, X, Y, T, G), axis=1).dropna(how='any', axis=0)

W = W.loc[tmp.index]
X = X.loc[tmp.index]
Y = Y.loc[tmp.index]
T = T.loc[tmp.index]
G = G.loc[tmp.index]

In [7]:
# Catboost を使用
model_y = CatBoostRegressor(loss_function="RMSE", verbose=0)
model_t = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="MultiClass",  # 学習時の指標は log-loss
    verbose=0
)

In [8]:
dml0 = LinearDML(model_y=model_y, model_t=model_t, discrete_treatment=True,
                 cv = GroupStratifiedKFold(n_splits=3)
                )
dml0.fit(Y.values.ravel(),
         T.values.ravel(),
         W=W.values,         
         groups=G.values, cache_values=True)

In [9]:
y_res, t_res, _, _ = dml0.residuals_

theta_hat = dml0.intercept_.ravel()        # shape (k,)
y_res     = y_res.ravel()                  # shape (n,)
t_res     = t_res                          # shape (n, k)

cluster_idx, clust_labels = pd.factorize(G)
num_clusters = clust_labels.size
n = len(y_res)
k = theta_hat.size

In [10]:
dml0.intercept__interval()

(array([ 0.00300061,  0.00817681, -0.0422321 ,  0.05078844]),
 array([0.05563851, 0.0809205 , 0.08515011, 0.24657902]))

In [11]:
theta_hat = dml0.intercept_.ravel()        # shape (k,)
y_res     = y_res.ravel()                  # shape (n,)
t_res     = t_res                          # shape (n, k)

cluster_idx, clust_labels = pd.factorize(G)
num_clusters = clust_labels.size
n = len(y_res)
k = theta_hat.size

# ── 1. ヤコビアン J_hat (k×k) ───────────────────────
J_hat = (t_res[:, :, None] * t_res[:, None, :]).mean(0)   # E[ t̃ t̃ᵀ ]
J_inv = np.linalg.inv(J_hat)

# ── 2. スコア ψ_i (n×k) ────────────────────────────
psi_i = t_res * (y_res - t_res.dot(theta_hat))[:, None]   # n × k
psi_i -= psi_i.mean(0, keepdims=True)                    # 再中心化

# ── 3. クラスタごとの合計 g_c ∈ ℝ^{k} ───────────────
cluster_sums = np.zeros((num_clusters, k))
for c in range(num_clusters):
    cluster_sums[c] = psi_i[cluster_idx == c].sum(0)

# ── 4. サンドイッチ分散 (k×k) ───────────────────────
B_hat = cluster_sums.T @ cluster_sums / n**2              # “meat”
cov_hat = J_inv @ B_hat @ J_inv.T                         # sandwich
se_hat = np.sqrt(np.diag(cov_hat))                       # (k,)

t_obs = theta_hat / se_hat                                # (k,)

# ── 5. クラスタ wild bootstrap (Rademacher) ──────────
n_boot = 100_000
rng = np.random.default_rng(450)
rad = np.array([-1, 1])
t_boot = np.empty((n_boot, k))

for b in range(n_boot):
    w_g = rng.choice(rad, size=num_clusters)              # ±1 per cluster
    psi_star = (w_g[:, None] * cluster_sums).sum(0) / n   # ψ̄*  (k,)
    theta_star = J_inv @ psi_star                         # bootstrap θ*
    t_boot[b] = theta_star / se_hat

# ── 6. 95% 信頼区間と p 値 ─────────────────────────
p_two = (np.abs(t_boot) >= np.abs(t_obs)).mean(0)         # 各係数の両側 p 値

q_lo, q_hi = np.quantile(t_boot, [0.95, 0.05], axis=0)  # パーセント点 (k,)
ci_lower = theta_hat - q_lo * se_hat
ci_upper = theta_hat - q_hi * se_hat

In [17]:
# ci_lower

In [18]:
# ci_upper

In [19]:
# dml0.summary()

In [20]:
# se_hat